# Tensorflow Hub Model with Ten Percent Data Augmenetation (Top 5 Diagnosis)

In [16]:
import pickle, random
import numpy as np

In [17]:
import pandas as pd

In [18]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [19]:
winogender_sentences = "all_sentences.tsv"

In [20]:
winogender_occupations = "occupations-stats.tsv"

In [21]:
import csv

wino_occupations = []

In [22]:
with open(winogender_occupations, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        wino_occupations.append(lines["occupation"])

In [23]:
# wino_occupations

In [24]:
len(wino_occupations)

60

In [25]:
len(set(wino_occupations))

60

### Load Text Classification Hub Model

In [26]:
import tensorflow as tf

In [27]:
rand_range = range(1, 11)

In [28]:
percent_mode = "ten"
path_start = "../saved_model/" + str(percent_mode) + "_percent_models/my_model_txt_classifier_hub_with_" + str(percent_mode) + "_percent_extra_data_rand_"
path_start

'../saved_model/ten_percent_models/my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_'

In [29]:
# my_model directory
# !ls ../saved_model

test_pth = "../saved_model/" + str(percent_mode)+ "_percent_models/"
!ls $test_pth

my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_1
my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_10
my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_2
my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_3
my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_4
my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_5
my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_6
my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_7
my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_8
my_model_txt_classifier_hub_with_ten_percent_extra_data_rand_9


In [30]:
# Contains an assets folder, saved_model.pb, and variables folder.
test_pth = "../saved_model/" + str(percent_mode)+ "_percent_models/my_model_txt_classifier_hub_with_" + str(percent_mode) + "_percent_extra_data_rand_1"
!ls $test_pth

assets            keras_metadata.pb saved_model.pb    variables


## Test on Repaired Input Tokens

### A. Direct gender noun comparisons (e.g. My boyfriend/My girlfriend)

In [31]:
mode = "direct-gender-noun"

In [32]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [33]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [34]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [35]:
noun_error = {**noun_error1, **noun_error2}

In [36]:
# noun_error 

In [37]:
noun_error1 = list(noun_error1)[:5]
noun_error2 = list(noun_error2)[:5]

In [38]:
noun_error1

['my aunt', 'my daughter', 'my mother', 'my wife', 'this girl']

In [39]:
noun_error2

['my uncle', 'my son', 'my father', 'my brother', 'my husband']

In [40]:
noun_error = get_sorted_dict(noun_error)

In [41]:
noun_error = list(noun_error)

In [42]:
input1, input2, input3 = [],[], []

In [43]:
input_pairs, input_truple = [], []

In [44]:
occupation_list = []

In [45]:
i = 0
tmp, tmp2 ="", ""

In [46]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [47]:
f_input, m_input, neut_input = "", "", ""

In [48]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [49]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [50]:
# fairness_errors = []
# all_inputs = []
# count = 0

In [51]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [52]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [53]:
len(all_inputs)

3000

In [54]:
len(fairness_errors)

105

In [55]:
len(fairness_errors_all_rands)

10

In [56]:
fairness_errors_all_rands

[248, 122, 127, 81, 57, 138, 216, 99, 130, 105]

In [57]:
sum(fairness_errors_all_rands)

1323

In [58]:
sum(fairness_errors_all_rands)/10

132.3

In [59]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [60]:
sum(all_rand_inputs)

30000

In [61]:
sum(all_rand_inputs)/10

3000.0

In [62]:
A_errors = sum(fairness_errors_all_rands)

In [63]:
A_inputs = sum(all_rand_inputs)

### B. Random gender noun comparisons (e.g. My boyfriend/My mother)

In [64]:
mode = "random-gender-noun"

In [65]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [66]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [67]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [68]:
noun_error = {**noun_error1, **noun_error2}

In [69]:
noun_error1 = list(noun_error1)[:5]
noun_error2 = list(noun_error2)[:5]

In [70]:
noun_error1

['my aunt', 'my girlfriend', 'my daughter', 'my wife', 'my mother']

In [71]:
noun_error2

['my brother', 'my father', 'this boy', 'my boyfriend', 'my uncle']

In [72]:
noun_error = get_sorted_dict(noun_error)

In [73]:
noun_error = list(noun_error)

In [74]:
input1, input2, input3 = [],[], []

In [75]:
input_pairs, input_truple = [], []

In [76]:
occupation_list = []

In [77]:
i = 0
tmp, tmp2 ="", ""

In [78]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [79]:
f_input, m_input, neut_input = "", "", ""

In [80]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = ""
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = ""
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [81]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [82]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [83]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [84]:
len(all_inputs)

3000

In [85]:
len(fairness_errors)

83

In [86]:
len(fairness_errors_all_rands)

10

In [87]:
fairness_errors_all_rands

[159, 85, 140, 95, 64, 119, 116, 74, 151, 83]

In [88]:
sum(fairness_errors_all_rands)

1086

In [89]:
sum(fairness_errors_all_rands)/10

108.6

In [90]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [91]:
sum(all_rand_inputs)

30000

In [92]:
sum(all_rand_inputs)/10

3000.0

In [93]:
B_errors = sum(fairness_errors_all_rands)

In [94]:
B_inputs = sum(all_rand_inputs)

### C. Test for Indirect Gender Bias, i.e. Occupational Bias

In [95]:
mode = "gender-occupation-noun"

In [96]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [97]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [98]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [99]:
noun_error = {**noun_error1, **noun_error2}

In [100]:
noun_error1 = list(noun_error1)[:5]
noun_error2 = list(noun_error2)[:5]

In [101]:
noun_error1

['the writer', 'the teacher', 'the cleaner', 'the auditor', 'the counselor']

In [102]:
noun_error2

['the cook', 'the analyst', 'the engineer', 'the mechanic', 'the technician']

In [103]:
noun_error = get_sorted_dict(noun_error)

In [104]:
noun_error = list(noun_error)

In [105]:
input1, input2, input3 = [],[], []

In [106]:
input_pairs, input_truple = [], []

In [107]:
occupation_list = []

In [108]:
i = 0
tmp, tmp2 ="", ""

In [109]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [110]:
f_input, m_input, neut_input = "", "", ""

In [111]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1

In [112]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [113]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [114]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [115]:
len(all_inputs)

3000

In [116]:
len(fairness_errors)

191

In [117]:
len(fairness_errors_all_rands)

10

In [118]:
fairness_errors_all_rands

[230, 205, 205, 157, 101, 249, 188, 188, 112, 191]

In [119]:
sum(fairness_errors_all_rands)

1826

In [120]:
sum(fairness_errors_all_rands)/10

182.6

In [121]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [122]:
sum(all_rand_inputs)

30000

In [123]:
sum(all_rand_inputs)/10

3000.0

In [124]:
C_errors = sum(fairness_errors_all_rands)

In [125]:
C_inputs = sum(all_rand_inputs)

### D. Test for   for Indirect Gender Bias, i.e. Name Bias

In [126]:
mode = "gender-name-noun"

In [127]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [128]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [129]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [130]:
noun_error = {**noun_error1, **noun_error2}

In [131]:
noun_error1 = list(noun_error1)[:5]
noun_error2 = list(noun_error2)[:5]

In [132]:
noun_error1

['Deborah', 'Sharon', 'Kimberly', 'Carol', 'Michelle']

In [133]:
noun_error2

['Christopher', 'Joshua', 'Thomas', 'James', 'Timothy']

In [134]:
noun_error = get_sorted_dict(noun_error)

In [135]:
noun_error = list(noun_error)

In [136]:
input1, input2, input3 = [],[], []

In [137]:
input_pairs, input_truple = [], []

In [138]:
occupation_list = []

In [139]:
i = 0
tmp, tmp2 ="", ""

In [140]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [141]:
f_input, m_input, neut_input = "", "", ""

In [142]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" 
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1

In [143]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [144]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [145]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [146]:
len(all_inputs)

3000

In [147]:
len(fairness_errors)

130

In [148]:
len(fairness_errors_all_rands)

10

In [149]:
fairness_errors_all_rands

[274, 219, 103, 90, 52, 177, 196, 127, 99, 130]

In [150]:
sum(fairness_errors_all_rands)

1467

In [151]:
sum(fairness_errors_all_rands)/10

146.7

In [152]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [153]:
sum(all_rand_inputs)

30000

In [154]:
sum(all_rand_inputs)/10

3000.0

In [155]:
D_errors = sum(fairness_errors_all_rands)

In [156]:
D_inputs = sum(all_rand_inputs)

### E. Test for   for Indirect Racial Bias, i.e. Name Bias

In [157]:
mode = "racial-name-noun"

In [158]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [159]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [160]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [161]:
noun_error = {**noun_error1, **noun_error2}

In [162]:
noun_error1 = list(noun_error1)[:5]
noun_error2 = list(noun_error2)[:5]

In [163]:
noun_error1

['Josh', 'Melanie', 'Justin', 'Stephanie', 'Harry']

In [164]:
noun_error2

['Malik', 'Lamar', 'Jasmine', 'Jamel', 'Nichelle']

In [165]:
noun_error = get_sorted_dict(noun_error)

In [166]:
noun_error = list(noun_error)

In [167]:
input1, input2, input3 = [],[], []

In [168]:
input_pairs, input_truple = [], []

In [169]:
occupation_list = []

In [170]:
i = 0
tmp, tmp2 ="", ""

In [171]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [172]:
f_input, m_input, neut_input = "", "", ""

In [173]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = ""
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [174]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [175]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [176]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [177]:
len(all_inputs)

3000

In [178]:
len(fairness_errors)

133

In [179]:
len(fairness_errors_all_rands)

10

In [180]:
fairness_errors_all_rands

[159, 162, 250, 130, 126, 159, 289, 79, 274, 133]

In [181]:
sum(fairness_errors_all_rands)

1761

In [182]:
sum(fairness_errors_all_rands)/10

176.1

In [183]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [184]:
sum(all_rand_inputs)

30000

In [185]:
sum(all_rand_inputs)/10

3000.0

In [186]:
E_errors = sum(fairness_errors_all_rands)

In [187]:
E_inputs = sum(all_rand_inputs)

### F. Test for Neutral (Sentiment) Sentences

In [188]:
mode = "neutral-sentiments"

In [189]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [190]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [191]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [192]:
noun_error = {**noun_error1, **noun_error2}

In [193]:
# noun_error 

In [194]:
noun_error1 = list(noun_error1)[:5]
noun_error2 = list(noun_error2)[:5]

In [195]:
noun_error1

['my aunt', 'my daughter', 'my wife', 'my mother', 'my girlfriend']

In [196]:
noun_error2

['my son', 'my uncle', 'my father', 'my husband', 'my brother']

In [197]:
noun_error = get_sorted_dict(noun_error)

In [198]:
noun_error = list(noun_error)

In [199]:
input1, input2, input3 = [],[], []

In [200]:
input_pairs, input_truple = [], []

In [201]:
occupation_list = []

In [202]:
i = 0
tmp, tmp2 ="", ""

In [203]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [204]:
f_input, m_input, neut_input = "", "", ""

In [205]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" 
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [206]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [207]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [208]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [209]:
len(all_inputs)

3000

In [210]:
len(fairness_errors)

63

In [211]:
len(fairness_errors_all_rands)

10

In [212]:
fairness_errors_all_rands

[238, 126, 171, 110, 59, 139, 174, 90, 126, 63]

In [213]:
sum(fairness_errors_all_rands)

1296

In [214]:
sum(fairness_errors_all_rands)/10

129.6

In [215]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [216]:
sum(all_rand_inputs)

30000

In [217]:
sum(all_rand_inputs)/10

3000.0

In [218]:
F_errors = sum(fairness_errors_all_rands)

In [219]:
F_inputs = sum(all_rand_inputs)

In [220]:
all_inputs = A_inputs + B_inputs + C_inputs + D_inputs + E_inputs + F_inputs

In [221]:
all_errors = A_errors + B_errors + C_errors + D_errors + E_errors + F_errors

In [222]:
all_inputs

180000

In [223]:
all_errors

8759

In [224]:
all_inputs/6

30000.0

In [225]:
all_errors/6

1459.8333333333333

In [226]:
all_errors_list = [A_errors, B_errors, C_errors, D_errors, E_errors, F_errors]

In [227]:
all_errors_list

[1323, 1086, 1826, 1467, 1761, 1296]

In [228]:
all_inputs_list = [A_inputs, B_inputs, C_inputs, D_inputs, E_inputs, F_inputs]

In [229]:
all_inputs_list

[30000, 30000, 30000, 30000, 30000, 30000]